In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os

from files import ImageFile
from processor import (GrayScaleProcessor, BlurProcessor, SobelProcessor, 
                       ThresholdProcessor, MorphologyProcessor, HistogramEqualizationProcessor,
                      CannyProcessor, ScaleProcessor, HoughLinesProcessor,
                      SaveImageProcessor, InvertProcessor, RetrieveImageProcessor)
from segment_processor import (HierarchyContourSegmenter, LargeSegmentFilter, 
                               SmallSegmentFilter, LargeAreaSegmentFilter, ContainedSegmentFilter, UniqueSegmentFilter,
                              RatioSegmentFilter)
from feature_extractor import (SimpleFeatureExtractor, ThreeChannelsFeatureExtractor)
from segment_classifier import DeepCnnSegmentClassifier, CnnSegmentClassifier, DeepCnnSigmoidSegmentClassifier, DeepBNCnnSegmentClassifier
from processor_stack import ProcessorStack
from power_number_reader import PowerNumberReader

import cv2
import itertools
import numpy as np
import time
adaptiveBlockSizes = [9, 17, 31]
#erodes = [3, 7]
#dilates = [3]
erodes = [(0, 0), (3, 3), (3, 3), (5, 5), (7, 7)]
dilates = [(0, 0),(3, 3), (7, 3), (5, 3), (7, 3)]
test_image= ImageFile('Brad meter 1')
#test_image= ImageFile('Brad meter 2')
#test_image= ImageFile('Davide take 1')
#test_image= ImageFile('Davide take 2')
#test_image= ImageFile('Erik') #Hard set adaptiveBlockSize=9 to catch digits
#test_image= ImageFile('Gro') #Impossible
#test_image= ImageFile('Hans Martin') #quite Hard
#test_image= ImageFile('Knut G')
#test_image= ImageFile('Ole auto')
#test_image= ImageFile('Ole blitz ikke')
#test_image= ImageFile('Ole blitz')
#test_image= ImageFile('Ole liggende')
#test_image= ImageFile('Roy') #small picture, need to scale
#test_image= ImageFile('Sven Erik close up')
#test_image= ImageFile('Sven Erik')
#test_image= ImageFile('Thomas') #Digital

#test_image= ImageFile('downloaded/139889743') 
#test_image= ImageFile('downloaded/LeckyMeterDavidSillitoe460') 

#test_image= ImageFile('downloaded/metric meter') #Hard

#test_image= ImageFile('downloaded/Electric-Meter') 
#test_image= ImageFile('downloaded/sangamo_meter') 
#test_image= ImageFile('test/meter1') 
#test_image= ImageFile('test/meter2') 
#test_image= ImageFile('test/4-cropped') #image too small issue 
#test_image = ImageFile('test/8Cmhx')
#test_image = ImageFile('test/40A_ferranti_meter')
#test_image = ImageFile('test/87bf22d8-4722-46f6-a76b-2c3775ceee14')

#test_image = ImageFile('test/106_6929')

#test_image = ImageFile('test/300px-ThreePhaseElectricityMeter')

#test_image = ImageFile('test/1112-water-meter-p')

#test_image = ImageFile('test/001422601')

#test_image = ImageFile('test/6024681_orig')
#test_image = ImageFile('test/electric-meter-dials-8523374')
#test_image = ImageFile('test/pl2217273-intelligent_small_din_rail_kwh_meter_digital_electricity_meter_1_phase_2_wire') #white bg
#test_image = ImageFile('test/Landis_&_Gyr_3_rate_electromechanical_meter._Type_CG1t')#3 lines
#test_image = ImageFile('test/single-phase-induction-kilo-watt-hour-meter')
def detect_digits(image):
    segments = np.empty( shape=(0, 0) )
    for adaptiveBlockSize, ((erode_x, erode_y), (dilate_x, dilate_y))  in itertools.product(adaptiveBlockSizes, zip(erodes, dilates)):
        digitDetectorStack = [ScaleProcessor(maxHeight=800, maxWidth=800),
             SaveImageProcessor(),
             GrayScaleProcessor(),
             BlurProcessor(blur_x=5, blur_y=5),
             HistogramEqualizationProcessor(HistogramEqualizationProcessor.CLAHE),
             ThresholdProcessor(ThresholdProcessor.ADAPTIVE, adaptiveBlockSize=adaptiveBlockSize), # 31 for normal. Change this to catch digits!
             MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(erode_x, erode_y), op = MorphologyProcessor.ERODE), #Change this
             MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(dilate_x, dilate_y), op = MorphologyProcessor.DILATE),
             HierarchyContourSegmenter(),
             RetrieveImageProcessor(),
             LargeSegmentFilter(min_width=4, min_height=13),
             SmallSegmentFilter(max_width=100, max_height=200),
             LargeAreaSegmentFilter(min_area = 120),
             RatioSegmentFilter(min_h_w_ratio = 1, max_h_w_ratio = 4.5)
             ] 
        digitDetector = ProcessorStack(digitDetectorStack)
        digitDetector.process(image)
        #digitDetector.display()
        output = digitDetector.get_output()
        if len(segments) == 0:
            segments = output['segments']
            image = output['originalImg']
        else:
            segments = np.concatenate((segments, output['segments']), axis = 0)
    
    print segments.shape
    print 'Done Segmentation!'
    
    segmentFilterStack = [UniqueSegmentFilter(threshold = 0.7),
                          ThreeChannelsFeatureExtractor(feature_size=32, extension = 4, #good number 4 pixel #Should set this as percentage of batch
                                                        method=ThreeChannelsFeatureExtractor.PAD_BACKGROUND ,display_features = False),
                          #CnnSegmentClassifier(trained_network_file='mode_cnn_5x5_430itr_manual_rotation_digit_detector.npz', n_output = 2),
                          #DigitRelevantSegmentFilter(),
                          #DeepCnnSegmentClassifier(trained_network_file='model_deep_cnn_3x3_30itr_new_svhn_manual_rotation.npz', n_output = 11),
                          #DeepCnnSigmoidSegmentClassifier(trained_network_file='model_deep_cnn_sigmoid_output_30itrs_svhn_manual_rotation.npz', n_output = 11),#good
                          
                          #DeepCnnSegmentClassifier(trained_network_file='model_deep_cnn_3x3_20itr_new_svhn_manual_rotation.npz', n_output = 2),
                          DeepBNCnnSegmentClassifier(trained_network_file='model_deep_BN_cnn_18itr_1.5Mdataset.npz', n_output = 11),
                          PowerNumberReader(bank_extension = 0, 
                                          bank_cut_off_ratio = 0.3, 
                                          height_error_acceptance = 0.7, 
                                          distance_error_acceptance = 0.3,
                                          residual_threshold = 0.085),
                         ]
    segmentFilter = ProcessorStack(segmentFilterStack)
    segmentFilter.process(image, segments = segments)
    return segmentFilter
    
test_image = cv2.imread('C:/Users/k/Google Drive/eSmart/Experiment/DigitRecognition/digit_recognition/Fail Images/201612-1454432228873.jpg')
detect_digits(test_image).display()
print 'Done'
continue_from_file = 'Davide take 2.jpg'
continue_from_file = 'Ole blitz.jpg'
continue_from_file = 'Ole liggende.jpg'

#continue_from_file = 'electricity-meter-picture.jpg'
#continue_from_file = 'electromechanical_meter_img.jpg'
#continue_from_file = 'ge-watt-hour-meter.jpg'

is_processed = False
for root, dirs, files in os.walk("data"):
    for file in files:
        print file
        if file.endswith(continue_from_file):
            is_processed = False
            print 'OK'
        if not is_processed:
            if file.lower().endswith(('.png','.tif','.jpg', '.jpeg')):
                start = time.time()
                image = cv2.imread(os.path.join(root, file))
                cv2.namedWindow( "Original", cv2.WINDOW_AUTOSIZE );
                cv2.imshow( "Original", image );       
                segmentFilter = detect_digits(image)
                end = time.time()
                print "Total time: ", end-start
                segmentFilter.display()

/home/nikasa/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN not available)


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
output = segmentFilter.get_output()
segments = output['segments']
segment_types = output['segment_types']
regions = output['regions']

from power_number_reader import PowerNumberReader
powerReaderStack = [PowerNumberReader(bank_extension = 0, 
                                      bank_cut_off_ratio = 0.8, 
                                      height_error_acceptance = 0.7, 
                                      distance_error_acceptance = 0.4),]
powerReader = ProcessorStack(powerReaderStack)
powerReader.process(image, segments = segments, segment_types = segment_types, regions = regions)
powerReader.display()

In [ ]:
np.savetxt("prediction.csv", prediction, delimiter=",")

In [ ]:
a = []
a.append(1)

In [ ]:
import numpy
xs = numpy.asarray([1,2,3, 4, 5, 7])
ys = numpy.asarray([1,3,2, 4, 5, 7])
a = numpy.polyfit(xs, ys, deg=1, full=True)
print(xs[xs>3])

In [ ]:
segmentFilter.display()


In [ ]:
zip([(1,2), (2,3)], [(3,2), (2, 1)])